In [1]:
import numpy as np
import pandas as pd
import datetime
from sklearn.cross_validation import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
import matplotlib

In [2]:
def getfullYears(date):
    date=str(date)
    return int(date.split("-")[0])+(float(date.split("-")[1])/12)
    
print getfullYears('2008-02-01')

2008.16666667


In [4]:
df = pd.read_csv('../data/aden_golfe_matrix_global.csv',sep=';')
print df.head()
listDate = df.apply(lambda row: datetime.datetime.strptime(row['date'], "%Y-%m-%d"), axis=1)
df['date']=listDate
df['Full_Years'] = df.apply(lambda row: getfullYears(row['date']), axis=1)
df['count'] = df.apply(lambda row: row['Attempted']+row['Boarded']+row['Fired upon']+row['Hijacked'], axis=1)
df=df.set_index('date')
print df.head()

df_global=df['count']
df_type=df[['Attempted' , 'Boarded' , 'Fired upon' , 'Hijacked']]

   Annee (0)  quinzaine_week  count
0       2008               3      2
1       2008               6      3
2       2008               7      3
3       2008               8      3
4       2008              10      6


KeyError: ('date', u'occurred at index 0')

In [5]:
# Time series prediction : version 1

In [6]:
def sub_plot_data(df, title="Number of attacks"):
    """Plot stock prices with a custom title and meaningful axis labels."""
    ax = df.plot(subplots=True,title=title, fontsize=12,figsize=(12,12))
    plt.show()
    
def plot_data(df, title="Number of attacks"):
    """Plot stock prices with a custom title and meaningful axis labels."""
    ax = df.plot(title=title, fontsize=12,figsize=(12,12))
    ax.set_xlabel("Date")
    ax.set_ylabel("Nb attacks")
    plt.show()



In [7]:
plot_data(df_global)

NameError: name 'df_global' is not defined

In [8]:
sub_plot_data(df_type)

NameError: name 'df_type' is not defined

In [9]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    
    #Determing rolling statistics
    rolmean = pd.rolling_mean(timeseries, window=12)
    rolstd = pd.rolling_std(timeseries, window=12)

    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Perform Dickey-Fuller test:
    print 'Results of Dickey-Fuller Test:'
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print dfoutput

In [10]:
test_stationarity(df_global)

NameError: name 'df_global' is not defined

In [11]:
test_stationarity(df_type['Attempted'])

NameError: name 'df_type' is not defined

In [12]:
test_stationarity(df_type['Boarded'])

NameError: name 'df_type' is not defined

In [13]:
test_stationarity(df_type['Fired upon'])

NameError: name 'df_type' is not defined

In [14]:
test_stationarity(df_type['Hijacked'])

NameError: name 'df_type' is not defined

In [15]:
def computedailyreturn(df):
    """ compute and return the daily return value"""
    dailyreturn=(df /df.shift(1))-1
    #print dailyreturn
    #dailyreturn.ix[0,:]=0
    return dailyreturn

In [16]:
daily=computedailyreturn(ts)

NameError: name 'ts' is not defined

In [17]:
daily.plot()

NameError: name 'daily' is not defined

In [18]:
# Time Series version 2

In [19]:
to_forecast = df_global.values

to_forecast_Attempted = df_type['Attempted'].values
to_forecast_Boarded = df_type['Boarded'].values
to_forecast_Fired = df_type['Fired upon'].values
to_forecast_Hijacked = df_type['Hijacked'].values

dates = df['Full_Years'].values
print df.head(5)

NameError: name 'df_global' is not defined

In [20]:
import numpy as np

def organize_data(to_forecast, window, horizon):
    """
     Input:
      to_forecast, univariate time series organized as numpy array
      window, number of items to use in the forecast window
      horizon, horizon of the forecast
     Output:
      X, a matrix where each row contains a forecast window
      y, the target values for each row of X
    """
    shape = to_forecast.shape[:-1] + (to_forecast.shape[-1] - window + 1, window)
    strides = to_forecast.strides + (to_forecast.strides[-1],)
    X = np.lib.stride_tricks.as_strided(to_forecast, shape=shape, strides=strides)
    y = np.array([X[i+horizon][-1] for i in range(len(X)-horizon)])
    return X[:-horizon], y

k = 13   # number of previous observations to use
h = 1  # forecast horizon
X,y = organize_data(to_forecast, k, h)

X_Attempted , y_Attempted = organize_data(to_forecast_Attempted, k, h)
X_Boarded , y_Boarded = organize_data(to_forecast_Boarded, k, h)
X_Fired , y_Fired = organize_data(to_forecast_Fired, k, h)
X_Hijacked , y_Hijacked = organize_data(to_forecast_Hijacked, k, h)


NameError: name 'to_forecast' is not defined

In [21]:
m = 55 # number of samples to take in account
def getPrediction(X,y):
    #regressor = LinearRegression(normalize=True)
    regressor=RandomForestRegressor()
    regressor.fit(X[:m], y[:m])
    predictions=regressor.predict(X)
    predictionError=regressor.predict(X[m:])
    predictions[predictions < 0] = 0
    
    return predictions,predictionError

predictionGlobal,predictionErrorGlobal=getPrediction(X,y)

prediction_Attempted , predictionError_Attempted = getPrediction(X_Attempted,y_Attempted )
prediction_Boarded , predictionError_Boarded = getPrediction(X_Boarded,y_Boarded)
predictionGlobal_Fired , predictionError_Fired = getPrediction(X_Fired,y_Fired)
predictionGlobal_Hijacked, predictionError_Hijacked = getPrediction(X_Hijacked,y_Hijacked)

NameError: name 'X' is not defined

In [22]:
def mape(ypred, ytrue):
    """ returns the mean absolute percentage error """
    ypred[ypred < 0] = 0
    idx = ytrue != 0.0
    return 100*np.mean(np.abs(ypred[idx]-ytrue[idx])/ytrue[idx])

print 'The error is %0.2f%%' % mape(predictionErrorGlobal,y[m:])

print 'The error is %0.2f%%' % mape(prediction_Attempted,y_Attempted[m:])
print 'The error is %0.2f%%' % mape(predictionError_Boarded,y_Boarded[m:])
print 'The error is %0.2f%%' % mape(predictionError_Fired,y_Fired[m:])
print 'The error is %0.2f%%' % mape(predictionError_Hijacked,y_Hijacked[m:])

NameError: name 'predictionErrorGlobal' is not defined

In [23]:
def plotPrediction(prediction,y,title):
    plt.figure(figsize=(10,8))
    plt.plot(listDate[k:],prediction, '--', color='#EB3737', linewidth=3, label='Prediction')
    plt.plot(listDate[k:k+m],y[:m], label='Train data', color='#3700B8', linewidth=2)
    plt.grid(True)
    plt.legend(loc='upper right')
    plt.title(title)
    plt.ylabel('number of attacks')
    plt.show()

plotPrediction(predictionGlobal,y,"Global")

plotPrediction(prediction_Attempted ,y_Attempted,"Attempted")
plotPrediction(prediction_Boarded,y_Boarded,"Boarded")
plotPrediction(predictionGlobal_Fired,y_Fired,"Fired")
plotPrediction(predictionGlobal_Hijacked,y_Hijacked,"Hijacked")

NameError: name 'predictionGlobal' is not defined

In [24]:
dfTest = pd.DataFrame(index=listDate)
prediction=np.concatenate([np.repeat(np.NAN,k),predictions])
true_value=np.concatenate([np.repeat(np.NAN,k),y[:m],np.repeat(np.NAN,len(listDate)-(k+m))])
dfTest['prediction']=prediction
dfTest['True']=true_value

NameError: name 'listDate' is not defined

In [25]:
dfTest.plot(figsize=(12,12),grid=True)

NameError: name 'dfTest' is not defined